In [2]:
from google.colab import files
uploaded = files.upload()

Saving black_friday_data_kaggle.csv to black_friday_data_kaggle.csv


In [0]:
import pandas as pd
import numpy as np

In [60]:

#get csv from url
df = pd.read_csv('black_friday_data_kaggle.csv', error_bad_lines=False)
df.head()

,Unnamed: 0,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
0,0,1000001,P00069042,F,0-17,10,A,2,0,-1,NaN,NaN,8370
1,1,1000001,P00248942,F,0-17,10,A,2,0,1,6.0,14.0,15200
2,2,1000001,P00087842,F,0-17,10,A,2,0,-1,NaN,NaN,1422
3,3,1000001,P00085442,F,0-17,10,A,2,0,-1,14.0,NaN,1057
4,4,1000002,P00285442,M,55+,16,C,4+,0,8,NaN,NaN,7969


In [0]:
# df.City_Category = pd.get_dummies(df.City_Category)
df.Gender = pd.get_dummies(df.Gender)
df.Age = pd.get_dummies(df.Age)
df.Stay_In_Current_City_Years = pd.get_dummies(df.Stay_In_Current_City_Years)
df.Marital_Status = pd.get_dummies(df.Marital_Status)
df.City_Category = pd.get_dummies(df.City_Category)

In [0]:

df_product = df.groupby(by='Product_ID', as_index=False).agg({'Purchase': lambda x: x.iloc[0], 'Age': lambda x: x.iloc[0], 'Marital_Status': lambda x: x.iloc[0], 'Stay_In_Current_City_Years': lambda x: x.iloc[0], 'Gender': np.mean, 'City_Category': lambda x: x.iloc[0], 'Occupation': lambda x: x.iloc[0], 'User_ID': pd.Series.nunique, 'Product_Category_1': lambda x: x.iloc[0]})

df_product['Log_Num_Users'] = np.log(df_product.User_ID)

In [64]:
print(df_product.shape)
df_product.head()

(3623, 11)


,Product_ID,Purchase,Age,Marital_Status,Stay_In_Current_City_Years,Gender,City_Category,Occupation,User_ID,Product_Category_1,Log_Num_Users
0,P00000142,13353,0,0,0,0.302655,0,12,1130,-1,7.029973
1,P00000242,10129,0,0,0,0.239892,0,12,371,-1,5.916202
2,P00000342,5201,0,1,1,0.281513,0,0,238,-1,5.472271
3,P00000442,5178,0,0,0,0.500000,0,5,92,-1,4.521789
4,P00000542,5211,0,1,1,0.342466,1,4,146,-1,4.983607


In [0]:
df_train = df_product[df_product.Product_Category_1 != -1]
df_test = df_product[df_product.Product_Category_1 == -1] # Not to be touched until the model is trained and ready

# Divide the training data into test and train data to measure accuracy
y = df_train['Product_Category_1']
X = df_train.drop('Product_Category_1', axis=1)

X = X.drop('Product_ID', axis=1)

In [67]:
df_test.head()

,Product_ID,Purchase,Age,Marital_Status,Stay_In_Current_City_Years,Gender,City_Category,Occupation,User_ID,Product_Category_1,Log_Num_Users
0,P00000142,13353,0,0,0,0.302655,0,12,1130,-1,7.029973
1,P00000242,10129,0,0,0,0.239892,0,12,371,-1,5.916202
2,P00000342,5201,0,1,1,0.281513,0,0,238,-1,5.472271
3,P00000442,5178,0,0,0,0.500000,0,5,92,-1,4.521789
4,P00000542,5211,0,1,1,0.342466,1,4,146,-1,4.983607


In [0]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [69]:
X_train.head()

,Purchase,Age,Marital_Status,Stay_In_Current_City_Years,Gender,City_Category,Occupation,User_ID,Log_Num_Users
2779,6159,0,0,0,0.500000,0,9,6,1.791759
1600,3624,0,0,1,0.247159,0,1,352,5.863631
2205,12570,0,1,0,0.261905,0,17,42,3.737670
1741,6063,0,0,0,0.224490,0,5,98,4.584967
1237,7036,1,1,0,0.666667,0,10,3,1.098612


In [70]:
from sklearn import tree

# clf = tree.DecisionTreeClassifier(max_depth=20)
# f = KFold(n_splits=8)
# f.get_n_splits(data)
  
clf = tree.DecisionTreeClassifier()
clf.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [71]:
Y_train_pred = clf.predict(X_train)

Y_train_pred

array([ 8,  5,  6, ..., 11,  2,  5])

In [0]:
from sklearn.metrics import accuracy_score

In [73]:
accuracy_score(y_train, Y_train_pred)

1.0

In [0]:
Y_test_pred = clf.predict(X_test)

In [75]:
accuracy_score(y_test, Y_test_pred)

0.7581453634085213

In [76]:
df_test = df_test.copy().drop(columns='Product_Category_1')

test_product_ids = df_test.Product_ID
df_test = df_test.drop('Product_ID', axis=1)

df_test.head()

,Purchase,Age,Marital_Status,Stay_In_Current_City_Years,Gender,City_Category,Occupation,User_ID,Log_Num_Users
0,13353,0,0,0,0.302655,0,12,1130,7.029973
1,10129,0,0,0,0.239892,0,12,371,5.916202
2,5201,0,1,1,0.281513,0,0,238,5.472271
3,5178,0,0,0,0.500000,0,5,92,4.521789
4,5211,0,1,1,0.342466,1,4,146,4.983607


In [0]:
Y_final = clf.predict(df_test)


In [78]:
Y_final.shape

(1207,)

In [79]:
Y_final

array([ 3,  3,  5, ...,  8, 10,  8])

In [80]:
df_test['Product_Category_1'] = Y_final

df_test.head()

,Purchase,Age,Marital_Status,Stay_In_Current_City_Years,Gender,City_Category,Occupation,User_ID,Log_Num_Users,Product_Category_1
0,13353,0,0,0,0.302655,0,12,1130,7.029973,3
1,10129,0,0,0,0.239892,0,12,371,5.916202,3
2,5201,0,1,1,0.281513,0,0,238,5.472271,5
3,5178,0,0,0,0.500000,0,5,92,4.521789,5
4,5211,0,1,1,0.342466,1,4,146,4.983607,5


In [0]:
df_test.to_csv("arjun.csv", sep=',')

In [82]:
pedro = pd.read_csv("pedro.csv")

pedro.head()

,Unnamed: 0,Occupation,Stay_In_Current_City_Years,Marital_Status,Product_Category_2,Product_Category_3,Purchase,Gender_F,Gender_M,Age_0-17,...,Age_26-35,Age_36-45,Age_46-50,Age_51-55,Age_55+,City_Category_A,City_Category_B,City_Category_C,User_ID,Product_Category_1
0,0,0.398628,0.448230,0.375221,0.125000,0.133333,0.460906,0.302655,0.697345,0.048673,...,0.385841,0.195575,0.071681,0.049558,0.022124,0.215929,0.379646,0.404425,0.607970,3.0
1,1,0.379111,0.479111,0.369272,0.125000,0.400000,0.436040,0.239892,0.760108,0.051213,...,0.350404,0.210243,0.067385,0.070081,0.018868,0.221024,0.477089,0.301887,0.199246,2.0
2,2,0.393487,0.444328,0.399160,0.750000,0.644656,0.215130,0.281513,0.718487,0.046218,...,0.390756,0.147059,0.054622,0.096639,0.063025,0.260504,0.411765,0.327731,0.127625,5.0
3,3,0.342391,0.451087,0.413043,0.375000,0.644656,0.193908,0.500000,0.500000,0.021739,...,0.500000,0.184783,0.043478,0.032609,0.021739,0.391304,0.413043,0.195652,0.049004,5.0
4,4,0.361986,0.503425,0.308219,0.490134,0.644656,0.220151,0.342466,0.657534,0.061644,...,0.376712,0.219178,0.047945,0.061644,0.006849,0.287671,0.472603,0.239726,0.078083,5.0


In [88]:
final = pedro[['Product_Category_1']]
final.rename(index=str, columns={"Product_Category_1":"pedro"})

final['arjun'] = Y_final
final['Product_ID'] = test_product_ids

final.index = final.Product_ID
final = final.drop(columns="Product_ID")
final.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


,Product_Category_1,arjun
Product_ID,,
P00000142,3.0,3
P00000242,2.0,3
P00000342,5.0,5
P00000442,5.0,5
P00000542,5.0,5


In [0]:
round_down_final = final